# Final Project

## Introduction

In the following development we will tend to reproduce numeralically the results published by Sussillo and Abbott onto their papers: 'Generating Coherent Patterns of Activity from Chaotic Neural Networks'.
In order to stuck to easily reproducable and understandable results, our results we be based on a complete coding of the learning algorithm avoiding the use of any previously coded librairies. 



## Paper abstract reminder

Neural circuits display complex activity patterns both spontaneously and when responding to a stimulus or generating a motor output. How are these two forms of activity related? We develop a procedure called FORCE learning for modifying synaptic strengths either external to or within a model neural network to change chaotic spontaneous activity into a wide variety of desired activity patterns. FORCE learning works even though the networks we train are spontaneously chaotic and we leave feedback loops intact and unclamped during learning. Using this approach, we construct networks that produce a wide variety of complex output patterns, input-output transformations that require memory, multiple outputs that can be switched by control inputs, and motor patterns matching human motion capture data. Our results reproduce data on premovement activity in motor and premotor cortex, and suggest that synaptic plasticity may be a more rapid and powerful modulator of network activity than generally appreciated.


## Results - (Sussillo and Abbott figure 2 reproduction)

In [4]:
import numpy as np
import matplotlib.pyplot as plt


In [5]:
np.random.seed(10645)

N_G = 1000 ### mauvaise valeur
g_GG = 1.3
p_GG = 0.1
p_z = 1
g_Gz = 1
g_GF = 0
alpha = 1
N_l = 0
tau = 10

In [6]:
def define_initial_weights(N_G, p_z):  
    """
    w_init = np.zeros(N_G)
    for k in range(N_G):
        p = np.random.uniform(0, 1, 1)
        if p <= p_z:
            w_init[k] = np.random.normal(0, 1/np.sqrt(p_z*N_G), 1)
     """
    return np.random.normal(0, np.sqrt(1/N_G), N_G)


def define_connection_matrix_J(N, proba):
    """
    J = np.array([np.zeros(N) for j in range(N)])
    for i in range(N):
        for j in range(N):
            p = np.random.uniform(0, 1, 1)
            if p  <= proba:
                J[i][j] = np.random.normal(0, 1/(proba*N), 1)
    """
    return np.random.normal(0,np.sqrt(1/(N*proba)), [N, N])

def update_x_t_by_euler_method(x, r, z, dt, tau, N_G, g_GG, J_GG, g_Gz, J_Gz):
    """ This function computes x(t+dt) knowing x(t) for every i
    This holds for Figure 2 and a simpler version of the differential equation
    x_in, r_in and z are functions of t ; tau, N_G, g_GG, J_GG, g_Gz and J_Gz are constants"""
    
    x_out = []
    
    for i in range(N_G):
        variation = -x[i] + g_GG * np.dot(J_GG[i], r) + g_Gz * J_Gz[i] * z
        variation = variation / tau
        x_out.append(x[i] + variation * dt)
    
    return np.array(x_out)


def compute_z(w, r):
    """ w and r as numpy arrays of same size. They are the weights and the 
    firing rates at time t. The function returns the dot
    product of the two, which is the output of the network)"""
    return np.dot(w, r)

def compute_error(w_delta, r, f):
    """ The function returns the error (scalar)"""
    return np.dot(w, r) - f

def check_decreasing_error(e_minus, e_plus):
    if abs(e_plus) <= abs(e_minus):
        return True
    return False

def modification_rule_weights(w, e_minus, P, r):
    """ w, e_minus and r are column arrays, P is a matrix 
    w is taken at time t - Delta_t ; the others at time t """
    w = w - e_minus * np.dot(P, r)
    return w

def modification_rule_matrix(P, r):
    """ P is a matrix, taken at time t - Delta_t
    r is the synaptic array taken at time t """
    D = np.outer(r, r)
    D = np.dot(D, P)
    D = np.dot(P, D)
    Pr = np.dot(P, r)
    a = np.dot(r, Pr)
    return P - D/(1 +a)





In [7]:
D = 4000 #duration in ms
dt = 0.05 #time step in ms
time = np.arange(0, D, dt)

def F1(t, period = 600):
    """ Sinus function"""
    return np.sin(2*np.pi*t/period)

def F2(t, period = 600):
    """ Fonction créneau"""
    if t%period < period/2:
        return -1
    return 1

def F3(t, period = 600):
    """ Fonction triangle"""
    t = t%period
    if t < period/2:
        return -1 + 2*t/300
    return 3 - 2*t/300

list_functions = [F1, F2, F3]

In [ ]:

J_GG = define_connection_matrix_J(N_G, p_GG)
J_Gz = np.random.uniform(-1, 1, N_G)

Fx=[]
Fy=[]
Z = np.zeros(int(D/dt))
F = np.zeros(int(D/dt))
list_ratio_errors = np.zeros(int(D/dt))
list_difference_errors_plus = np.zeros(int(D/dt))


# Define initial variables
P = np.eye(N_G)/alpha
w = define_initial_weights(N_G, p_z)
x = np.random.uniform(0, 1, N_G)
r = np.tanh(x)
z = compute_z(w,r)

# Before updating the weigths
W=[[],[],[],[]]
X=[[],[],[],[]]



for F in list_functions[:1]:
    
    mod = 1
    init=True
    finish=True
    for i in range (0,len(time)):
        t=time[i]
        f=0
        e_minus = compute_error(w, r, f)
        if t>D/4 and t<3*D/4 and t%10==0:
            if init==True:
                print('End of the spontaneaous activity')
                init=False
            Fx.append(t)
            f=F(t)
            Fy.append(f)
            P = modification_rule_matrix(P, r)
            e_minus = compute_error(w, r, f)
            w = modification_rule_weights(w, e_minus, P, r)
        if t - mod/10*D > 0:
            print('*', end = " ")
            mod+=1
        if t>3*D/4 and t%10==0 and finish==True:
            print('End of the learning phasis')
            finish=False
    
        e_minus = compute_error(w, r, f)
    
        x = update_x_t_by_euler_method(x, r, z, dt, tau, N_G, g_GG, J_GG, g_Gz, J_Gz)
    
        r = np.tanh(x)
        z = compute_z(w,r)
    
        Z[i]=(z)
    
    
        for i in range(0,4):
            W[i].append(w[i])
            X[i].append(x[i])

        Pr = np.dot(P, r)
        a = np.dot(r, Pr)
        e_plus = e_minus*(1-a)

        #list_ratio_errors.append(e_plus/e_minus)
        list_difference_errors_plus[i]=e_plus

print('*')
    


* * End of the spontaneaous activity
* * * * 

In [ ]:

def settings_all_plots():
    """ Rather than rewriting this everytime"""
    plt.axvline(D/4, -2, 2, color = 'gray', linewidth = 0.7)
    plt.axvline(3*D/4, -2, 2, color = 'gray', linewidth = 0.7)
    plt.xlabel('Time (ms)')
    plt.ylabel('Relative scale')
    plt.legend()

residu=[]
for sortie,target in zip(Z,F):
    residu.append(np.abs(sortie-target))

# Main plot
plt.figure(figsize = (16, 4))
plt.plot(time,Z, color = 'red', label = 'Output')
plt.plot(Fx,Fy, color = 'blue', label = 'Target')
#plt.plot(time,residu, label='residu')
plt.title('Evolution of the network output over time')
settings_all_plots()
plt.show()


# Plot the activity of four specific weights
plt.figure(figsize = (16, 4))
for i in range(0,4):
    plt.plot(W[i] + 2*i, label = 'Weight #' + str(i))

plt.yticks([])
settings_all_plots()
plt.show()

# Plot the activity of four specific activities
plt.figure(figsize = (16,4))
for i in range(0,5):
    plt.plot(X[i] + 2*i, label = 'Neuron #' + str(i))

settings_all_plots()
plt.yticks([])
plt.show()




In [ ]:
print(F[1900:2000])

## Examples of FORCE Learning (Sussillo and Abbott figure 3 reproduction)